# 🔬 QEPC Enhanced Backtest - FIXED

This notebook:
1. Uses **actual game results** from your data files
2. Compares QEPC predictions to real outcomes
3. Calculates detailed accuracy metrics
4. Generates visualizations

---

## 🔧 Setup

In [4]:
# SETUP - This replaces the broken 'from notebook_context import *'
import sys
from pathlib import Path

# Find project root (works from any notebook location)
current = Path.cwd()
project_root = None

for p in [current] + list(current.parents):
    if (p / "data").exists() and (p / "qepc").exists():
        project_root = p
        break

if project_root is None:
    project_root = current.parent.parent
    
print(f"📁 Project root: {project_root}")

# Add to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Standard imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization imports
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_PLOTS = True
    print("✅ Matplotlib loaded")
except ImportError:
    HAS_PLOTS = False
    print("⚠️ Matplotlib not available")

# QEPC imports - CORRECTED PATHS
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths
from qepc.core.lambda_engine import compute_lambda
from qepc.core.simulator import run_qepc_simulation

# Paths
data_dir = project_root / "data"
raw_dir = data_dir / "raw"

print("✅ Setup complete!")

📁 Project root: C:\Users\wdors\qepc_project
✅ Matplotlib loaded
✅ Setup complete!


---
## 📊 Load Actual Game Results

In [6]:
# Load game data
print("📊 Looking for game data...")

possible_paths = [
    raw_dir / "TeamStatistics.csv",
    data_dir / "TeamStatistics.csv",
    data_dir / "GameResults_2025.csv",
    data_dir / "Games.csv",
]

team_stats = None
for path in possible_paths:
    if path.exists():
        try:
            team_stats = pd.read_csv(path)
            print(f"✅ Loaded: {path.name} ({len(team_stats):,} rows)")
            break
        except Exception as e:
            print(f"⚠️ Error with {path.name}: {e}")

if team_stats is None:
    print("❌ No game data found!")
    print(f"   Searched: {[p.name for p in possible_paths]}")
else:
    # Parse dates
    date_col = None
    for col in ['gameDate', 'Date', 'date', 'GAME_DATE']:
        if col in team_stats.columns:
            date_col = col
            break
    
    if date_col:
        team_stats['gameDate'] = pd.to_datetime(team_stats[date_col], errors='coerce')
        
        # Drop rows with invalid dates
        valid_dates = team_stats['gameDate'].notna()
        invalid_count = (~valid_dates).sum()
        
        if invalid_count > 0:
            print(f"⚠️ Dropped {invalid_count} rows with invalid dates")
            team_stats = team_stats[valid_dates].copy()
        
        if len(team_stats) > 0:
            print(f"\n📅 Date range: {team_stats['gameDate'].min().date()} to {team_stats['gameDate'].max().date()}")
        else:
            print("❌ No valid dates in data!")
    else:
        print(f"⚠️ No date column found")
    
    print(f"\n📋 Columns available: {list(team_stats.columns)[:10]}...")

📊 Looking for game data...
✅ Loaded: TeamStatistics.csv (144,314 rows)
⚠️ Dropped 143758 rows with invalid dates

📅 Date range: 2025-10-02 to 2025-11-17

📋 Columns available: ['gameDate', 'teamCity', 'teamName', 'opponentTeamCity', 'opponentTeamName', 'teamScore', 'opponentScore', 'reboundsTotal', 'assists', 'threePointersMade']...


---
## 🎯 Set Backtest Parameters

In [7]:
if team_stats is not None and 'gameDate' in team_stats.columns:
    # Auto-detect date range from data
    latest_date = team_stats['gameDate'].max()
    earliest_date = team_stats['gameDate'].min()
    
    # Default: last 30 days of available data
    BACKTEST_START = latest_date - timedelta(days=30)
    BACKTEST_END = latest_date
    
    print(f"🎯 Backtest Configuration:")
    print(f"   Start: {BACKTEST_START.date()}")
    print(f"   End:   {BACKTEST_END.date()}")
    print(f"   Days:  {(BACKTEST_END - BACKTEST_START).days}")
    
    # Filter to backtest window
    backtest_data = team_stats[
        (team_stats['gameDate'] >= BACKTEST_START) &
        (team_stats['gameDate'] <= BACKTEST_END)
    ].copy()
    
    # Get home games only (avoid duplicates)
    if 'home' in backtest_data.columns:
        backtest_games = backtest_data[backtest_data['home'] == 1].copy()
    else:
        backtest_games = backtest_data.copy()
    
    print(f"\n📊 Games in backtest window: {len(backtest_games)}")
    
    # Create standardized team name columns
    if 'teamName' in backtest_games.columns:
        backtest_games['Home_Team'] = (backtest_games.get('teamCity', '') + ' ' + backtest_games['teamName']).str.strip()
        backtest_games['Away_Team'] = (backtest_games.get('opponentTeamCity', '') + ' ' + backtest_games.get('opponentTeamName', '')).str.strip()
    
    # Create score columns
    for src, dst in [('teamScore', 'Home_Score'), ('opponentScore', 'Away_Score')]:
        if src in backtest_games.columns:
            backtest_games[dst] = backtest_games[src]
    
    if len(backtest_games) > 0:
        print("✅ Ready to backtest!")
    else:
        print("❌ No games found in date range")
else:
    print("❌ Cannot set parameters - no data loaded")

🎯 Backtest Configuration:
   Start: 2025-10-18
   End:   2025-11-17
   Days:  30

📊 Games in backtest window: 207
✅ Ready to backtest!


---
## 🚀 Run QEPC Predictions

In [8]:
print("🔮 Running QEPC predictions...\n")

results = []
errors_log = []

if 'backtest_games' in dir() and len(backtest_games) > 0:
    total_games = len(backtest_games)
    
    for i, (idx, game) in enumerate(backtest_games.iterrows()):
        # Progress indicator
        if (i + 1) % 10 == 0 or i == 0:
            print(f"⏳ Processing game {i+1}/{total_games}...", end="\r")
        
        try:
            home_team = game.get('Home_Team', game.get('teamName', 'Home'))
            away_team = game.get('Away_Team', game.get('opponentTeamName', 'Away'))
            
            # Get team strengths
            strengths = calculate_advanced_strengths(verbose=False)
            
            if strengths.empty:
                errors_log.append(f"Game {i}: No strength data")
                continue
            
            # Build schedule
            schedule = pd.DataFrame([{
                'Home Team': home_team,
                'Away Team': away_team
            }])
            
            # Compute lambdas
            schedule_with_lambda = compute_lambda(schedule, strengths)
            
            # Run simulation
            predictions = run_qepc_simulation(schedule_with_lambda, num_trials=5000)
            
            if len(predictions) == 0:
                continue
            
            pred = predictions.iloc[0]
            
            # Get predictions
            pred_home = pred.get('Sim_Home_Score', pred.get('lambda_home', 110))
            pred_away = pred.get('Sim_Away_Score', pred.get('lambda_away', 108))
            home_win_prob = pred.get('Home_Win_Prob', 0.5)
            
            # Get actuals
            actual_home = game.get('Home_Score', game.get('teamScore', 0))
            actual_away = game.get('Away_Score', game.get('opponentScore', 0))
            
            # Calculate outcomes
            actual_home_won = actual_home > actual_away
            pred_home_won = home_win_prob > 0.5
            
            results.append({
                'Date': game['gameDate'],
                'Home_Team': home_team,
                'Away_Team': away_team,
                'Pred_Home_Score': round(pred_home, 1),
                'Pred_Away_Score': round(pred_away, 1),
                'Pred_Total': round(pred_home + pred_away, 1),
                'Pred_Spread': round(pred_home - pred_away, 1),
                'Home_Win_Prob': round(home_win_prob, 3),
                'Actual_Home_Score': actual_home,
                'Actual_Away_Score': actual_away,
                'Actual_Total': actual_home + actual_away,
                'Actual_Spread': actual_home - actual_away,
                'Winner_Correct': actual_home_won == pred_home_won,
                'Error_Total': abs((pred_home + pred_away) - (actual_home + actual_away)),
                'Error_Spread': abs((pred_home - pred_away) - (actual_home - actual_away)),
            })
            
        except Exception as e:
            errors_log.append(f"Game {i}: {str(e)[:40]}")
    
    print("\n")  # Clear progress line
    
    results_df = pd.DataFrame(results)
    print(f"✅ Backtest complete!")
    print(f"   Games analyzed: {len(results_df)}")
    print(f"   Errors skipped: {len(errors_log)}")
else:
    print("❌ No games to backtest")

🔮 Running QEPC predictions...

⏳ Processing game 200/207...

✅ Backtest complete!
   Games analyzed: 0
   Errors skipped: 207


---
## 📈 Analyze Results

In [9]:
if 'results_df' in dir() and len(results_df) > 0:
    # Calculate metrics
    win_accuracy = results_df['Winner_Correct'].mean()
    avg_total_error = results_df['Error_Total'].mean()
    avg_spread_error = results_df['Error_Spread'].mean()
    median_total_error = results_df['Error_Total'].median()
    
    print("="*60)
    print("📊 BACKTEST RESULTS")
    print("="*60)
    print(f"""
┌─────────────────────────────────────────────────┐
│              PERFORMANCE SUMMARY                │
├─────────────────────────────────────────────────┤
│  Games Analyzed:     {len(results_df):>6}                    │
│  Win Accuracy:       {win_accuracy:>6.1%}                    │
├─────────────────────────────────────────────────┤
│  Avg Total Error:    {avg_total_error:>6.1f} pts               │
│  Median Total Error: {median_total_error:>6.1f} pts               │
│  Avg Spread Error:   {avg_spread_error:>6.1f} pts               │
└─────────────────────────────────────────────────┘
    """)
    
    # High confidence analysis
    results_df['Confidence'] = abs(results_df['Pred_Spread'])
    high_conf = results_df[results_df['Confidence'] > 5]
    
    if len(high_conf) > 0:
        print(f"\n🎯 High Confidence Games (|spread| > 5):")
        print(f"   Count: {len(high_conf)}")
        print(f"   Accuracy: {high_conf['Winner_Correct'].mean():.1%}")
    
    # Best predictions
    print(f"\n🏆 Best Predictions (smallest error):")
    best = results_df.nsmallest(5, 'Error_Total')
    for _, row in best.iterrows():
        date = pd.Timestamp(row['Date']).strftime('%m-%d')
        print(f"   {date}: {row['Away_Team'][:18]:18} @ {row['Home_Team'][:18]:18} | Error: {row['Error_Total']:.1f}")
    
    # Worst predictions
    print(f"\n⚠️ Worst Predictions (largest error):")
    worst = results_df.nlargest(5, 'Error_Total')
    for _, row in worst.iterrows():
        date = pd.Timestamp(row['Date']).strftime('%m-%d')
        print(f"   {date}: {row['Away_Team'][:18]:18} @ {row['Home_Team'][:18]:18} | Error: {row['Error_Total']:.1f}")
    
    print("\n" + "="*60)
else:
    print("❌ No results to analyze")

❌ No results to analyze


---
## 📊 Visualizations

In [10]:
if HAS_PLOTS and 'results_df' in dir() and len(results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Predicted vs Actual Total
    ax1 = axes[0, 0]
    ax1.scatter(results_df['Actual_Total'], results_df['Pred_Total'], alpha=0.6, s=50)
    min_val = min(results_df['Actual_Total'].min(), results_df['Pred_Total'].min()) - 10
    max_val = max(results_df['Actual_Total'].max(), results_df['Pred_Total'].max()) + 10
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect')
    ax1.set_xlabel('Actual Total', fontsize=12)
    ax1.set_ylabel('Predicted Total', fontsize=12)
    ax1.set_title('Predicted vs Actual Total Score', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Error Distribution
    ax2 = axes[0, 1]
    ax2.hist(results_df['Error_Total'], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
    ax2.axvline(results_df['Error_Total'].mean(), color='r', linestyle='--', linewidth=2, 
                label=f'Mean: {avg_total_error:.1f}')
    ax2.axvline(results_df['Error_Total'].median(), color='orange', linestyle='--', linewidth=2,
                label=f'Median: {median_total_error:.1f}')
    ax2.set_xlabel('Total Error (points)', fontsize=12)
    ax2.set_ylabel('Frequency', fontsize=12)
    ax2.set_title('Distribution of Total Score Error', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Error Over Time
    ax3 = axes[1, 0]
    results_sorted = results_df.sort_values('Date')
    ax3.plot(range(len(results_sorted)), results_sorted['Error_Total'], 
             marker='o', alpha=0.6, markersize=5, linewidth=1)
    ax3.axhline(avg_total_error, color='r', linestyle='--', linewidth=2, label='Mean Error')
    ax3.set_xlabel('Game Number', fontsize=12)
    ax3.set_ylabel('Total Error (points)', fontsize=12)
    ax3.set_title('Prediction Error Over Time', fontsize=14)
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Win Accuracy by Confidence
    ax4 = axes[1, 1]
    bins = [0, 3, 6, 10, 100]
    labels = ['0-3', '3-6', '6-10', '10+']
    results_df['Conf_Bin'] = pd.cut(results_df['Confidence'], bins=bins, labels=labels)
    
    accuracy_by_conf = results_df.groupby('Conf_Bin', observed=True)['Winner_Correct'].agg(['mean', 'count'])
    
    bars = ax4.bar(range(len(accuracy_by_conf)), accuracy_by_conf['mean'], color='steelblue')
    ax4.axhline(0.5, color='r', linestyle='--', linewidth=2, label='50% (coin flip)')
    ax4.axhline(win_accuracy, color='green', linestyle='--', linewidth=2, label=f'Overall: {win_accuracy:.1%}')
    ax4.set_xticks(range(len(accuracy_by_conf)))
    ax4.set_xticklabels(labels)
    ax4.set_xlabel('Predicted Spread (confidence)', fontsize=12)
    ax4.set_ylabel('Win Accuracy', fontsize=12)
    ax4.set_title('Win Accuracy by Confidence Level', fontsize=14)
    ax4.set_ylim(0, 1)
    ax4.legend()
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Add count labels on bars
    for i, (bar, count) in enumerate(zip(bars, accuracy_by_conf['count'])):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                 f'n={int(count)}', ha='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
    print("✅ Visualizations complete")
elif not HAS_PLOTS:
    print("⚠️ Matplotlib not available - skipping visualizations")
else:
    print("❌ No data to visualize")

❌ No data to visualize


---
## 💾 Save Results

In [ ]:
if 'results_df' in dir() and len(results_df) > 0:
    # Save detailed results
    output_dir = project_root / "data" / "results" / "backtests"
    output_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Enhanced_Backtest_{timestamp}.csv"
    output_path = output_dir / filename
    
    results_df.to_csv(output_path, index=False)
    print(f"💾 Saved results to: {output_path}")
    
    # Print final summary
    print(f"""
📋 FINAL SUMMARY
================
Period:       {BACKTEST_START.date()} to {BACKTEST_END.date()}
Games:        {len(results_df)}
Win Accuracy: {win_accuracy:.1%}
Avg Error:    {avg_total_error:.1f} pts
    """)
else:
    print("❌ No results to save")

print("🏁 Backtest complete!")

---
## 🎯 Next Steps

### Based on your results:

**If Win Accuracy < 55%:**
- Add recency weighting to team strengths
- Include rest day adjustments
- Consider injuries impact

**If Total Error > 15 points:**
- Calibrate lambda calculations
- Add pace adjustments
- Review team volatility modeling

**If High Confidence games underperform:**
- Add upset probability (quantum tunneling)
- Consider travel factors
- Review matchup-specific adjustments

---

**Use these insights to improve QEPC!** 🚀